In [ ]:
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/fishial/Fishial/Object-Detection-Model')
from module.segmentation_package.src.utils import get_sorted_data
import os
import cv2
import copy
import torch
import time
import logging
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A

from os import listdir
from os.path import isfile, join

from module.segmentation_package.src.utils import run_eval_checkpoints
from module.segmentation_package.src.utils import get_current_date_in_format
from module.segmentation_package.src.utils import get_dataset_dicts
from module.segmentation_package.src.utils import get_prepared_data, get_empty_ann
from module.segmentation_package.src.utils import remove_tmp_files
from module.segmentation_package.src.utils import save_json, read_json
from module.segmentation_package.src.utils import split_ds
from module.segmentation_package.src.trainer import Trainer

import albumentations as A

from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
import cv2

# copy paste source
from module.segmentation_package.src.copy_paste import CopyPaste
from module.segmentation_package.src.coco import CocoDetectionCP
from module.segmentation_package.src.CopyPasteCustom import apply_copy_paste_aug, get_copy_paste_instance, get_images_from_instance
from detectron2.data import build_detection_train_loader
from pycocotools import mask
from skimage import measure

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultTrainer
from detectron2.projects.point_rend import ColorAugSSDTransform, add_pointrend_config
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultPredictor

from module.segmentation_package.interpreter_segm import SegmentationInference
import detectron2
print(detectron2.__path__)

def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
output_path = r'/home/fishial/Fishial/output/citiscape/18_11_2022_14_28_56/coco_eval.json'
data = read_json(output_path)

In [ ]:
for i in data:
    ap = data[i]['segm']['AP']
    ap50 = data[i]['segm']['AP50']
    ap75 = data[i]['segm']['AP75']
    print(i,ap, ap50, ap75)

In [ ]:
config_path = "/home/fishial/Fishial/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml"

In [ ]:
cfg = get_cfg()
add_pointrend_config(cfg)

cfg.merge_from_file(config_path)
cfg.MODEL.POINT_HEAD.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = "/home/fishial/Fishial/output/citiscape/23_11_2022_14_50_20/model_1649999.pth"
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
cfg.MODEL.DEVICE = 'cuda'
# cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 128
# cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 64
# cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 64

model = DefaultPredictor(cfg)

In [ ]:
path_to_empty_image_folder = r'/home/fishial/Fishial/dataset/coco_val2017'
file_images = [f for f in listdir(path_to_empty_image_folder) if isfile(join(path_to_empty_image_folder, f))]

count_false_positive = 0
delta_time = []
scores = []
for idx, filename in enumerate(file_images):
    path = join(path_to_empty_image_folder, filename)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    s_t = time.time()
    output = model(img)
    
    delta_time.append(time.time() - s_t)
    count_false_positive += len(output['instances'].pred_classes)
    scores.extend(output['instances'].scores.detach().cpu().numpy())
    print(f"idx: {idx} count: {count_false_positive} average time: {Average(delta_time)}", end='\r')

In [ ]:
print(cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE)
print(cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN)

In [ ]:
/home/fishial/Fishial/saved_models/point_rend_top_fish.pth = 2847
/home/fishial/Fishial/output/NEW_DATASET/10_11_2022_11_58_02/model_1999999.pth = 1138 / 1129
/home/fishial/Fishial/output/NEW_DATASET/07_11_2022_10_32_26/model_1999999.pth = 1228
/home/fishial/Fishial/output/NEW_DATASET/07_11_2022_10_32_26/model_0999999.pth = 1927
/home/fishial/Fishial/output/amp_on/02_11_2022_11_44_11/model_0999999.pth = 1536
/home/fishial/Fishial/output/amp_on/02_11_2022_11_44_11/model_0859999.pth = 1557
/home/fishial/Fishial/output/amp_on/31_10_2022_20_33_47/model_0999999.pth = 2361
/home/fishial/Fishial/output/amp_on/31_10_2022_20_33_47/model_0499999.pth = 2392
/home/fishial/Fishial/output/citiscape/16_11_2022_09_58_04/model_1049999.pth = 1586
/home/fishial/Fishial/output/citiscape/16_11_2022_09_58_04/model_1649999.pth = 1677
/home/fishial/Fishial/output/citiscape/18_11_2022_11_00_37/model_0049999.pth = 8265
            
/home/fishial/Fishial/output/citiscape/18_11_2022_14_28_56/model_1999999.pth = 1571
/home/fishial/Fishial/output/citiscape/18_11_2022_14_28_56/model_0649999.pth = 2277
/home/fishial/Fishial/output/citiscape/22_11_2022_14_35_57/model_0099999.pth = 2003
/home/fishial/Fishial/output/citiscape/23_11_2022_14_50_20/model_1049999.pth = 1170
/home/fishial/Fishial/output/citiscape/23_11_2022_14_50_20/model_1299999.pth = 1194/1194
/home/fishial/Fishial/output/citiscape/23_11_2022_14_50_20/model_1799999.pth = 1162



In [ ]:
commutes = pd.Series(np.array(scores))

commutes.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
plt.title('histogram')
plt.xlabel('count')
plt.ylabel('score')
plt.grid(axis='y', alpha=0.75)